# Binomial Option Pricing Model
---

In [1]:
import math
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import compiler

OPT_TYPE_CALL = 0
OPT_TYPE_PUT = 1

## 변수설정
---

In [2]:
# 옵션 타입: 콜=OPT_TYPE_CALL, 풋=OPT_TYPE_PUT
opt_type = OPT_TYPE_CALL

# 기초자산 가격
spot_prc = 274.34

# 행사가
strike_prc = 275.0

# 잔존일
remain_day = 31

# 무위험 이자율
interest_rate = 1.55

# 변동성
volatility = 13.59

# 노드갯수
node_cnt = 49

## Black-Scholes Model
---

In [3]:
def get_d1(s, k, t, r, v):
	return (math.log(s / k) + (r + v * v * 0.5) * t) / (v * math.sqrt(t))


def get_d2(s, k, t, r, v):
	return (math.log(s / k) + (r - v * v * 0.5) * t) / (v * math.sqrt(t))


def get_nm_cdf(x):
	a1 = 0.31938153
	a2 = -0.356563782
	a3 = 1.781477937
	a4 = -1.821255978
	a5 = 1.330274429

	abs_x = math.fabs(x)
	k = 1.0 / (1.0 + 0.2316419 * abs_x)
	n_x = 1.0 - (math.exp(-1*abs_x*abs_x*0.5) / math.sqrt(2*math.pi)) * (a1*k + a2*k*k + a3*k*k*k + a4*k*k*k*k + a5*k*k*k*k*k)

	ret = 0

	if 0 > x:
		ret = 1.0 - n_x
	else:
		ret = n_x

	return ret


def get_bs_opt_prc(o_type, s, k, t, r, v):
	d1 = get_d1(s, k, t, r, v)
	d2 = get_d2(s, k, t, r, v)

	if OPT_TYPE_CALL == o_type:
		opt_prc = s * get_nm_cdf(d1) - k * math.exp(-1 * r * t) * get_nm_cdf(d2)
	else:
		opt_prc = k * math.exp(-1 * r * t) * get_nm_cdf(-1 * d2) - s * get_nm_cdf(-1 * d1)

	return opt_prc


bs_opt_prc = get_bs_opt_prc(
    opt_type,
    spot_prc,
    strike_prc,
    remain_day / 365,
    interest_rate * 0.01,
    volatility * 0.01
)

print("옵션가", bs_opt_prc)

옵션가 4.189248806121185


## Binomial Option Pricing Model (CPU)
---

In [4]:
st_time = time.time()

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

datas = [0] * (node_cnt + 1)

for row in range(node_cnt+1):
    datas[row] = spot_prc * (u ** (node_cnt - 2 * row))
    datas[row] = max(datas[row]-strike_prc, 0) if OPT_TYPE_CALL == opt_type else max(strike_prc-datas[row], 0)

for col in range(node_cnt, 0, -1):
    for row in range(col):
        datas[row] = (p * datas[row] + r_p * datas[row+1]) / a

ed_time = time.time()

print("소요시간(초)", ed_time - st_time)
print("옵션가", datas[0])

소요시간(초) 0.0009996891021728516
옵션가 4.203350070605942


## Binomial Option Pricing Model (GPU)
---

In [5]:
# Kernel code
kernel_code = """
__constant__ int int_params[2];
__constant__ double double_params[7];

__global__ void get_w_avg_prc(double* l_vals, double* r_vals) {
	const int node_cnt = int_params[0];
	const double a = double_params[2];
	const double p = double_params[5];
	const double r_p = double_params[6];

	const int idx = threadIdx.x + blockIdx.x * blockDim.x;

	if ( node_cnt > idx ) {
		l_vals[idx] = (p * r_vals[idx] + r_p * r_vals[idx + 1]) / a;
	}
}

__global__ void get_payoffs(double* payoffs) {
	const int node_cnt = int_params[0];
	const int opt_type = int_params[1];
	const double spot_prc = double_params[0];
	const double strike_prc = double_params[1];
	const double u = double_params[3];

	const int idx = threadIdx.x + blockIdx.x * blockDim.x;

	const double final_spot_prc = spot_prc * pow(u, (double)(node_cnt - 2 * idx));
	const double payoff = (0==opt_type) ? fmax(final_spot_prc-strike_prc, 0.0) : fmax(strike_prc-final_spot_prc, 0.0);

	payoffs[idx] = payoff;
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

st_time = time.time()

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

params = [spot_prc, strike_prc, a, u, d, p, r_p]

inputs_int = np.array([node_cnt, opt_type], dtype=np.int32)
inputs_double = np.array(params, dtype=np.float64)

# Constant 메모리에 값 복사
drv.memcpy_htod(mod.get_global("int_params")[0], inputs_int)
drv.memcpy_htod(mod.get_global("double_params")[0], inputs_double)

# Get kernel function
get_w_avg_prc = mod.get_function("get_w_avg_prc")
get_payoffs = mod.get_function("get_payoffs")

# Numpy array를 생성
host_input_memory1 = np.zeros(node_cnt + 1, dtype=np.float64)
host_input_memory2 = np.zeros(node_cnt + 1, dtype=np.float64)

# GPU 메모리 할당
device_memory1 = drv.mem_alloc(host_input_memory1.nbytes)
device_memory2 = drv.mem_alloc(host_input_memory2.nbytes)

# Host에서 Device로 메모리 복사
drv.memcpy_htod(device_memory1, host_input_memory1)
drv.memcpy_htod(device_memory2, host_input_memory2)

# Thread와 Block 갯수 계산
thread_cnt = 32
block_cnt = math.ceil(host_input_memory1.size / thread_cnt)

# Payoff들을 계산
get_payoffs(device_memory1, block=(thread_cnt, 1, 1), grid=(block_cnt, 1, 1))

# 가중 평균 계산
i = node_cnt
input_flag = True

while 0 < i:
	block_cnt = math.ceil(i / thread_cnt)

	if input_flag:
		get_w_avg_prc(device_memory2, device_memory1, block=(thread_cnt, 1, 1), grid=(block_cnt, 1, 1))
	else:
		get_w_avg_prc(device_memory1, device_memory2, block=(thread_cnt, 1, 1), grid=(block_cnt, 1, 1))

	input_flag = not input_flag
	i = i - 1

# 결과값을 Device에서 Host로 메모리 복사
opt_prc = np.zeros(1, dtype=np.float64)

if input_flag:
	drv.memcpy_dtoh(opt_prc, device_memory1)
else:
	drv.memcpy_dtoh(opt_prc, device_memory2)

ed_time = time.time()

print("소요시간(초)", ed_time - st_time)
print("옵션가", opt_prc[0])

소요시간(초) 0.004010677337646484
옵션가 4.203350070605941
